In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import torch
from torch.nn import DataParallel
from torch.utils.data import DataLoader
from torch.optim import AdamW

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
import gc

import sys
sys.path.append("../src")
import FE
import dataset
import models
import util
from config import read_config, updateConfig
import train_helper
import infer_helper
import loss
import lrfinder

In [3]:
print(torch.version.cuda)

11.1


# Config

In [4]:
BaseConfig = read_config("base")
{key: val for key, val in BaseConfig.__dict__.items() if not key.startswith('__')}

{'kaggle_data_folder': '/home/vincent/Kaggle/data/ventilator-pressure-prediction',
 'input_folder': '../input/',
 'output_folder': '../output/',
 'debug': False,
 'model_version': 'BasicTest',
 'model_module': 'Fork',
 'PL_folder': None,
 'seed': 48,
 'ckpt_folder': None,
 'use_lr_finder': False,
 'low_q': 0.05,
 'high_q': 0.95,
 'unit_var': True,
 'strict_scale': True,
 'hidden': [512, 256, 128, 64],
 'bidirectional': True,
 'nh': 256,
 'do_prob': 0.1,
 'epochs': 20,
 'es': 10,
 'train_folds': [0],
 'batch_size': 1024,
 'lr': 0.001,
 'weight_decay': 0.0001,
 'warmup': 0.1,
 'scheduler': 'cosineWithWarmUp',
 'use_swa': False,
 'use_wandb': False,
 'wandb_project': 'GBVPP',
 'wandb_key_path': '../input/key.txt',
 'wandb_post': '',
 'print_num_steps': 100,
 'num_workers': 8,
 'use_dp': True}

In [5]:
class config(BaseConfig):
    model_version = "debug"
    debug=False
    epochs = 300
    batch_size = 512
    lr=5e-2
    train_folds = [0]
    use_wandb = True
    use_lr_finder = False
config = updateConfig(config)

Using device: cuda
Number of device: 2
Model Output Folder: ../output/debug/


In [6]:
print(torch.cuda.is_available())

True


In [7]:
train, test = dataset.read_data(config)
display(train.shape, test.shape)
display(train.head())
display(test.head())

(102400, 9)

(102400, 7)

,id,breath_id,R,C,time_step,u_in,u_out,pressure,fold
0,1,1,20,50,0.000000,0.083334,0,5.837492,0
1,2,1,20,50,0.033652,18.383041,0,5.907794,4
2,3,1,20,50,0.067514,22.509277,0,7.876254,2
3,4,1,20,50,0.101542,22.808823,0,11.742872,0
4,5,1,20,50,0.135756,25.355850,0,12.234987,2


,id,breath_id,R,C,time_step,u_in,u_out
0,1,0,5,20,0.000000,0.000000,0
1,2,0,5,20,0.031904,7.515046,0
2,3,0,5,20,0.063827,14.651675,0
3,4,0,5,20,0.095751,21.230610,0
4,5,0,5,20,0.127644,26.320955,0


# adding features

In [8]:
train = FE.add_features(train)
test = FE.add_features(test)

In [9]:
feature_cols = [col for col in train.columns if col not in ["id", "breath_id", "fold", "pressure"]]
print(feature_cols)

['time_step', 'u_in', 'u_out', 'step', 'area', 'u_in_cumsum', 'time_step_diff', 'cross_time', 'cross_u_in', 'u_in_diff', 'u_in_diff_2', 'u_in_diff_5', 'ewm_u_in_mean', 'ewm_u_in_std', 'ewm_u_in_corr', 'rolling_10_mean', 'rolling_10_max', 'rolling_10_std', 'expand_mean', 'expand_median', 'expand_std', 'expand_max', 'expand_skew', 'expand_kurt', 'RC_u_in_median_diff', 'RC_u_in_mean_diff', 'R_20', 'R_5', 'R_50', 'C_10', 'C_20', 'C_50']


In [10]:
display(train.head()), display(test.head())

,id,breath_id,time_step,u_in,u_out,pressure,fold,step,area,u_in_cumsum,...,expand_skew,expand_kurt,RC_u_in_median_diff,RC_u_in_mean_diff,R_20,R_5,R_50,C_10,C_20,C_50
0,1,1,0.000000,0.083334,0,5.837492,0,0,0.000000,0.083334,...,0.000000,0.000000,-0.356609,-15.875539,1,0,0,0,0,1
1,2,1,0.033652,18.383041,0,5.907794,4,1,0.618632,18.466375,...,0.000000,0.000000,10.864344,-3.509825,1,0,0,0,0,1
2,3,1,0.067514,22.509277,0,7.876254,2,2,2.138332,40.975655,...,0.000000,0.000000,13.477262,3.459379,1,0,0,0,0,1
3,4,1,0.101542,22.808823,0,11.742872,0,3,4.454391,63.784477,...,3.227848,3.227848,13.416943,9.729695,1,0,0,0,0,1
4,5,1,0.135756,25.355850,0,12.234987,2,4,7.896588,89.140327,...,3.809552,3.809552,10.317239,5.620728,1,0,0,0,0,1


,id,breath_id,time_step,u_in,u_out,step,area,u_in_cumsum,time_step_diff,cross_time,...,expand_skew,expand_kurt,RC_u_in_median_diff,RC_u_in_mean_diff,R_20,R_5,R_50,C_10,C_20,C_50
0,1,0,0.000000,0.000000,0,0,0.000000,0.000000,0.033430,0.000000,...,0.000000,0.000000,-6.665972,-17.809572,0,1,0,0,1,0
1,2,0,0.031904,7.515046,0,1,0.239758,7.515046,0.031904,0.031904,...,0.000000,0.000000,-12.614708,-22.210510,0,1,0,0,1,0
2,3,0,0.063827,14.651675,0,2,1.174935,22.166721,0.031924,0.063827,...,0.000000,0.000000,-7.058000,-15.490029,0,1,0,0,1,0
3,4,0,0.095751,21.230610,0,3,3.207788,43.397331,0.031924,0.095751,...,-1.218656,-1.218656,-1.205360,-8.300224,0,1,0,0,1,0
4,5,0,0.127644,26.320955,0,4,6.567489,69.718285,0.031893,0.127644,...,-1.280772,-1.280772,3.673914,-3.933735,0,1,0,0,1,0


(None, None)

# Learning Rate Finder

In [11]:
if config.use_lr_finder:
    # data
    rs = RobustScaler(quantile_range=(config.low_q, config.high_q), unit_variance=config.unit_var)
    X_train = rs.fit_transform(train[feature_cols])
    X_train = X_train.reshape(-1, 80, len(feature_cols))
    y_train = train['pressure'].values.reshape(-1, 80)
    w_train = (1 - train['u_out']).values.reshape(-1, 80)
    train_dt = dataset.LR_VPP(X_train, y_train, w_train, config.batch_size)
    train_loader = DataLoader(train_dt,
                              batch_size=config.batch_size,
                              shuffle=True,
                              num_workers=config.num_workers, pin_memory=True, drop_last=False)

In [12]:
if config.use_lr_finder:
    # model
    model = models.Model(len(feature_cols),config)
    model.to(config.device)
    if config.use_dp:
        model = DataParallel(model)
    optimizer = AdamW(model.parameters(), lr=1e-6, eps=1e-08, weight_decay=config.weight_decay, amsgrad=False)
    criterion = loss.cal_mae_loss

In [13]:
if config.use_lr_finder:
    lr_finder = lrfinder.LRFinder(model, optimizer, criterion, config.device)
    lrs, losses = lr_finder.range_test(train_loader, end_lr=10, num_iter=100, smooth_f=0.05, diverge_th=5)    
    lrfinder.plot_lr_finder(lrs, losses)

In [15]:
if config.use_lr_finder:
    del model, optimizer, criterion, train_dt, train_loader, lr_finder
    gc.collect()
    torch.cuda.empty_cache() 

# Training

In [21]:
train_helper.training_loop(train.copy(), config)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/vincent/.netrc


Fold:  0
Use scale to fit train and scale valid
Prepare train valid
(82000, 36) (20400, 36)
training data samples, val data samples:  (1025, 80, 32) (255, 80, 32)
Model Size: 5690725
Epoch:  0
Save Model
loss:  17.5453, val_loss 7.9719, val_score 7.9719, best_val_score 7.9719, time used: 1.026s
Fold:  1
skip
Fold:  2
skip
Fold:  3
skip
Fold:  4
skip
folds score: [7.971910424202299]
Avg: 7.97191
Std: 0.00000


[fold0] avg_train_loss,▁
[fold0] avg_val_loss,▁
[fold0] epoch,▁
[fold0] loss,▅▁█
[fold0] lr,█▁█
[fold0] val_score,▁
[fold0] avg_train_loss,17.54532
[fold0] avg_val_loss,7.97191
[fold0] epoch,1
[fold0] loss,23.18336
[fold0] lr,0.025


In [17]:
gc.collect()
torch.cuda.empty_cache()

# Inference

In [22]:
test_avg = infer_helper.get_test_avg(test, config)

100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.17it/s]


count    1.024000e+05
mean     9.999978e-01
std      7.522517e-07
min      9.999932e-01
25%      9.999974e-01
50%      9.999976e-01
75%      9.999983e-01
max      9.999993e-01
Name: pressure, dtype: float64
test file saved to: ../output/debug//submission.csv
